In [15]:
from vnstock import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ffn
import vnstock
import bt
from backtesting import Backtest, Strategy
from backtesting.test import SMA
from backtesting.lib import crossover
%matplotlib inline

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [20]:
# Define parameters for stock screening insights
params = {"exchangeName": "HOSE,HNX,UPCOM"}

# Get stock screening insights data
df = stock_screening_insights(params, size=1700, drop_lang='vi')

# Merge with listing companies data
listing_df = listing_companies()
df = df.merge(listing_df, how='left', on='ticker')

# Filter for stocks in VN30 index
df = df[df['VN100']==True]

# Filter for stocks meeting conditions: Price above SMA(50), Price above SMA(100), rsi14 between 30 and 75
df = df[(df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)') (df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)')]


# Filter for stocks meeting additional conditions: epsGrowth5Year > 0, revenueGrowth5Year > 0,
# pe < 20, marketCap < 300000, rsi14 between 30 and 75, evEbitda >= 0, and financialHealth > 3.0
df = df[(df['pe'] < 20) & (df['rsi14'] >= 30) & (df['rsi14'] <= 70) &(df['stockRating'] >= 3)]

# Display the filtered DataFrame
df

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,...,VNCOND,VNCONS,VNENE,VNFIN,VNHEAL,VNIND,VNIT,VNMAT,VNREAL,VNUTI
12,ACB,,106229.0,24.8,3.1,4.3,3.6,3.6,0.1,0.8,...,False,False,False,True,False,False,False,False,False,False
120,BMP,,9308.0,39.2,3.1,3.8,3.0,5.0,0.2,0.9,...,False,False,False,False,False,True,False,False,False,False
277,CTG,,186607.0,17.1,3.0,3.6,4.5,3.6,0.1,0.9,...,False,False,False,True,False,False,False,False,False,False
311,DCM,,18794.0,10.8,3.0,2.2,4.4,5.0,0.1,1.4,...,False,False,False,False,False,False,False,True,False,False
482,GAS,,189022.0,18.7,3.1,2.4,3.6,5.0,0.0,0.8,...,False,False,False,False,False,False,False,False,False,True
502,GMD,,24472.0,28.7,3.3,3.8,4.4,4.6,0.2,1.1,...,False,False,False,False,False,True,False,False,False,False
552,HDG,,9112.0,12.3,3.0,3.2,3.0,3.4,-0.1,1.2,...,False,False,False,False,False,True,False,False,False,False
684,IMP,,4662.0,15.0,3.0,4.0,3.0,4.8,0.1,0.4,...,False,False,False,False,True,False,False,False,False,False
702,KBC,,25792.0,11.7,3.3,4.0,3.9,4.8,0.0,1.6,...,False,False,False,False,False,False,False,False,True,False
801,MBB,,124095.0,24.5,3.4,4.7,4.5,3.4,0.1,0.9,...,False,False,False,True,False,False,False,False,False,False


In [28]:
vnm_up = vnstock.stock_historical_data('VNM','2024-02-02','2024-03-13','1D')

In [29]:
# Đổi tên cột 'Time' thành 'Date'
def processing(df):
    df = df.rename(columns={'time': 'Date'})
    df = df.rename(columns={'open': 'Open'})
    df = df.rename(columns={'high': 'High'})
    df = df.rename(columns={'low': 'Low'})
    df = df.rename(columns={'close': 'Close'})
    df = df.rename(columns={'volume': 'Volume'})
    df = df.rename(columns={'ticker': 'Ticker'})

    # Chuyển định dạng cột 'Date' thành tháng/ngày/năm
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%m/%d/%Y')

    # Sắp xếp lại các cột
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in cols:
        df[col] = df[col] / 1000
    return df

In [30]:
vnm_up=processing(vnm_up)
vnm_up.to_csv('vnm_up.csv',index=False)

In [31]:
data1 = pd.read_csv('vnm_up.csv')

In [32]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 2)
        self.ma2 = self.I(SMA, price, 5)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(data1, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
bt.plot()
stats

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11188\4121802142.py:14: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data1, SmaCross, commission=.002,


Start                                     0.0
End                                      23.0
Duration                                 23.0
Exposure Time [%]                        75.0
Equity Final [$]                   9575.66348
Equity Peak [$]                   10120.70284
Return [%]                          -4.243365
Buy & Hold Return [%]                5.049594
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.139358
Avg. Drawdown [%]                   -8.139358
Max. Drawdown Duration                   17.0
Avg. Drawdown Duration                   17.0
# Trades                                  6.0
Win Rate [%]                        16.666667
Best Trade [%]                       2.572777
Worst Trade [%]                      -2.18754
Avg. Trade [%]                    

# X

In [50]:
def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

The strategy roughly goes like this:

Buy a position when:
* weekly RSI(30) $\geq$ daily RSI(30) $>$ 70
* Close $>$ MA(10) $>$ MA(20) $>$ MA(50) $>$ MA(100)

Close the position when:
* Daily close is more than 2% _below_ MA(10)
* 8% fixed stop loss is hit

We need to provide bars data in the _lowest time frame_ (i.e. daily) and resample it to any higher time frame (i.e. weekly) that our strategy requires.

In [51]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 2)
        self.ma20 = self.I(SMA, self.data.Close, 5)
        self.ma50 = self.I(SMA, self.data.Close, 10)
        self.ma100 = self.I(SMA, self.data.Close, 20)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [56]:
df = pd.read_csv('ACB.csv')
df.drop(columns=['Ticker'], inplace=True)

# Đặt cột 'Date' làm chỉ số (index)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [57]:
backtest = Backtest(df, System, commission=.002)
backtest.run()

Start                     2006-11-21 00:00:00
End                       2024-03-13 00:00:00
Duration                   6322 days 00:00:00
Exposure Time [%]                    2.574212
Equity Final [$]                  7143.221547
Equity Peak [$]                  15213.812713
Return [%]                         -28.567785
Buy & Hold Return [%]              363.078104
Return (Ann.) [%]                   -1.946896
Volatility (Ann.) [%]                7.193188
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -53.152951
Avg. Drawdown [%]                   -21.82353
Max. Drawdown Duration     6216 days 00:00:00
Avg. Drawdown Duration     2082 days 00:00:00
# Trades                                    9
Win Rate [%]                        22.222222
Best Trade [%]                      19.164656
Worst Trade [%]                    -14.676822
Avg. Trade [%]                    

In [58]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

  0%|          | 0/18 [00:00<?, ?it/s]

CPU times: total: 13.7 s
Wall time: 13.7 s


Start                     2006-11-21 00:00:00
End                       2024-03-13 00:00:00
Duration                   6322 days 00:00:00
Exposure Time [%]                   21.150278
Equity Final [$]                 55849.370106
Equity Peak [$]                  58458.120106
Return [%]                         458.493701
Buy & Hold Return [%]              363.078104
Return (Ann.) [%]                   10.574991
Volatility (Ann.) [%]               15.535063
Sharpe Ratio                         0.680718
Sortino Ratio                        1.227019
Calmar Ratio                         0.348402
Max. Drawdown [%]                  -30.352842
Avg. Drawdown [%]                    -4.98437
Max. Drawdown Duration     2010 days 00:00:00
Avg. Drawdown Duration      140 days 00:00:00
# Trades                                   15
Win Rate [%]                        73.333333
Best Trade [%]                      73.285276
Worst Trade [%]                    -12.284789
Avg. Trade [%]                    

In [59]:
backtest.plot()

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p5239', ...)